In [1]:
import pandas as pd
import numpy as np

## Задание 1

In [2]:
#читаю csv-файл
h_file = pd.read_csv('horse_data.csv')

#отбираю нужные столбцы
horse = h_file[['surgery?', 'Age', 'rectal temperature','pulse',
                'respiratory rate', 'temperature of extremities', 'pain', 'outcome']]

#изменяю '?' на NaN
horse = horse.replace('?', np.NaN)

#исправляю типы данных в столбцах
horse['surgery?'] = horse['surgery?'].astype('float64')
horse['rectal temperature'] = horse['rectal temperature'].astype('float64')
horse['pulse'] = horse['pulse'].astype('float64')
horse['respiratory rate'] = horse['respiratory rate'].astype('float64')
horse['temperature of extremities'] = horse['temperature of extremities'].astype('float64')
horse['pain'] = horse['pain'].astype('float64')
horse['outcome'] = horse['outcome'].astype('float64')

In [3]:
horse.info()
horse.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


## Поиск выбросов

In [4]:
# Столбцы surgery?, Age, temperature of extremities, pain, outcome имеют дискретные значения, которые не отличаются от 
# возможных. Поэтому в них выбросов - нет

print(horse['surgery?'].unique())
print(horse['Age'].unique()) #в столбце Age вместо ожидаемого значаения 2,
                            #стоит - 9. Принимаем, что произошла ошибка и 9 - это 2
print(horse['temperature of extremities'].unique())
print(horse['pain'].unique())
print(horse['outcome'].unique())

[ 2.  1. nan]
[1 9]
[ 3. nan  1.  4.  2.]
[ 5.  3.  2. nan  4.  1.]
[ 2.  3.  1. nan]


In [5]:
q1 = horse['rectal temperature'].quantile(0.25)
q3 = horse['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)

print(lower_bound)
print(upper_bound)

#выбросы есть, так как есть значения ниже и выше вычисленных значений

36.74999999999999
39.550000000000004


In [6]:
q1 = horse['pulse'].quantile(0.25)
q3 = horse['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)

print(lower_bound)
print(upper_bound)

#выбросы есть, так как есть значения выше вычисленного значения верхней границы

-12.0
148.0


In [7]:
q1 = horse['respiratory rate'].quantile(0.25)
q3 = horse['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)

print(lower_bound)
print(upper_bound)

#выбросы есть, так как есть значения выше вычисленного значения верхней границы

-7.75
62.25


## Работа с пропусками

In [8]:
#Заполнить пропуск surgery? метрикой - 1, так как боль была сильная,
#а значит,с большой долей вероятнсти, операция была проведена
horse['surgery?'].fillna( 1 , inplace=True)

# Заменяю пропуски rectal temperature медианным значением, так как у большинства лошадей завышена температура,
# в следствии болезни. Но из-за шокового состояния может снижаться на некоторые значения.
# Не ставлю среднее арифметическое, так как есть выбросы в данном столбце
horse['rectal temperature'].fillna(horse['rectal temperature'].median(), inplace=True)

# Заменить пропуски respiratory rate медианой, так как документации написано,
# что из-за больших колебаний метрика сомнительна
horse['respiratory rate'].fillna((round(horse['respiratory rate'].median())), inplace=True)

# Заменить пропуски temperature of extremities в соответсвии с температурой и пульсом.
# Температура конечностей коррелирует с ректальной температурой, при условии, что она повышена.
# При шоке (повышенный пульс), конечности будут холодными
# Так как у большинства лошадей пульс повышен, а медиана temperature of extremities = 3, заполняю пропуски значением 3
horse['temperature of extremities'].fillna(3, inplace=True)

# Замена пропусков pain в соответствии с была ли проведена операция, так как между этими столбцами есть зависимость.
# Если боль сильная и/или продолжительная, то нужна операция
horse['pain'].fillna(horse.groupby('surgery?')['pain'].transform('median'), inplace=True)

#Заполнить пропуск outcome значением - 1(lived), так как температура и пульс были завышены незначительно,
#и была проведена операция
horse['outcome'].fillna(1 , inplace=True)

#Заменяю пропуски pulse на медианное значение (повышенный пульс), так как у большинства лошадей замечена такая тенденция
horse['pulse'].fillna(horse['pulse'].median(), inplace=True)

In [9]:
horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    300 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB
